# CKAN MANAGER


In [11]:
import ckanapi
import re
import os
import requests
from bs4 import BeautifulSoup

In [12]:
# CKAN API base URL and API KEY
"""WARNING: ESTO NO HAGAS PUSH"""
CKAN_BASE_URL = "https://data.integratedmodelling.org/"
API_KEY = ""

ckan = ckanapi.RemoteCKAN(CKAN_BASE_URL, apikey=API_KEY)

In [13]:
def get_csv_file_list(path):
    File_list = [] #f for f in os.listdir(path) if os.isfile(mypath,f)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            if file not in File_list:
                File_list.append(os.path.join(path,file))
        else:
            pass
    return File_list

def get_all_datasets(ckan):
    try:
        datasets = ckan.action.package_list()
        all_datasets = []
        for dataset_id in datasets:
            dataset = ckan.action.package_show(id=dataset_id)
            all_datasets.append(dataset)
        return all_datasets
    except Exception as e:
        exceptions(e)
        return None

def get_dataset_by_name(ckan, dataset_name):
    try:
        result = ckan.action.package_search(q='name:"{}"'.format(dataset_name))
        datasets = result['results']
        if result['count'] == 0:
            print("Dataset with name '{}' does not exist.".format(dataset_name))
            return None
        elif result['count'] == 1:
            print("Dataset with name '{}' already exists.".format(dataset_name))
            return datasets[0]
        else:
            return print("Dataset with name '{}' are more than one.".format(dataset_name))
    except Exception as e:
        exceptions(e)
        return None
    
def get_organization_groups(ckan):
    try:
        organization_groups = ckan.action.organization_list(all_fields=True)
        return organization_groups
    except Exception as e:
        exceptions(e)
        return None
    
def get_all_licenses(ckan):
    try:
        licenses = ckan.action.license_list()
        return licenses
    except Exception as e:
        exceptions(e)
        return None

def create_dataset(ckan, dataset_metadata):
    try:
        created_dataset = ckan.action.package_create(**dataset_metadata)
        print("Create dataset: ", created_dataset["name"])
        return created_dataset
    except Exception as e:
        exceptions(e)
        return None

def create_resource(ckan, resource_metadata, file_obj=None):
    try:
        if file_obj:
            created_resource = ckan.action.resource_create(**resource_metadata, upload=file_obj)
            print("Create resource: ", created_resource["name"])
        else:
            created_resource = ckan.action.resource_create(**resource_metadata)
            print("Create resource: ", created_resource["name"])
        return created_resource
    except Exception as e:
        exceptions(e)
        return None
    
def create_view(ckan, view_params):
    try:
        created_view = ckan.action.resource_view_create(**view_params)
        return created_view
    except Exception as e:
        exceptions(e)
        return None

def delete_dataset(ckan, dataset_id):
    try:
        ckan.action.package_delete(id=dataset_id)
        return print("Dataset deleted successfully.")
    except Exception as e:
        exceptions(e)
        return None

def exceptions(exception):
    if isinstance(exception, ckanapi.NotFound):
        print("Not found:", exception)
    elif isinstance(exception, ckanapi.NotAuthorized):
        print("Not authorized:", exception)
    elif isinstance(exception, ckanapi.CKANAPIError):
        print("CKAN API error:", exception)
    else:
        print("An unexpected error I did not code occurred:", exception)

def get_url_xml(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print("Error: Failed to retrieve GetCapabilities document")
        return None
    
def extract_ows_layers(capabilities_xml, regex_pattern):
    soup = BeautifulSoup(capabilities_xml, 'xml')
    if soup.find('wcs:CoverageId'):
        layers = soup.find_all('wcs:CoverageId')
    elif soup.find('wfs:Name'):
        layers = soup.find_all('wfs:Name')
    else:
        print("wtf")

    layer_names = [layer.text for layer in layers]
    matched_layers = [layer for layer in layer_names if re.match(regex_pattern, layer)]
    return matched_layers

def describecoverage_or_feature_request(ows_url, layers):
    created_layers = []
    for layer in layers:
        if "__" in layer:
            describecoverage = r"?service=WCS&version=2.0.1&request=DescribeCoverage&CoverageId="
            created_layers.append(ows_url + describecoverage + layer)
        elif ":" in layer:
            describefeaturetype = r"?request=DescribeFeatureType&version=1.0.0&service=WFS&typeName="
            created_layers.append(ows_url + describefeaturetype + layer)
    return created_layers


def size_reduzer(width_str, height_str):
    width = int(width_str)
    height = int(height_str)
    max_value = max(width, height)       
    if max_value > 5000:
        # Lets scale it. I set the limit in 5000
        scale_factor = 5000 / max_value
        width = round(width * scale_factor)
        height = round(height * scale_factor)
        return str(width), str(height)
    else:
        return str(width), str(height)

def getmap_from_describecoverage(ows_url, describe_coverage):
    soup = BeautifulSoup(describe_coverage, 'xml')
    coverageid = soup.find('wcs:CoverageId').text.replace("__", "%3A")
    lowercorner = '%2C'.join(reversed(soup.find('gml:lowerCorner').text.split())) # I reverted the coord order values to fit the query
    uppercorner = '%2C'.join(reversed(soup.find('gml:upperCorner').text.split())) # I reverted the coord order values to fit the query
    original_width, original_height = soup.find("gml:high").text.split() # This is the original, but is too big
    
    width, height = size_reduzer(original_width, original_height)

    epsg = soup.find('gml:Envelope')['srsName'].split('/')[-1] # Got the EPSG from the XML
   
    view_layer = ows_url + "?service=WMS&version=1.1.0&request=GetMap&layers=" + coverageid + "&bbox=" + lowercorner + "%2C" + uppercorner + "&width=" + width + "&height=" + height + "&srs=EPSG%3A" + epsg + "&format=image%2Fjpeg"
    return view_layer

def getcoverage_request(ows_url, coverageid):
    getcoverage = ows_url + r"?request=GetCoverage&service=WCS&Version=2.0.1&coverageid=" + coverageid + r"&format=image/geotiff"
    return getcoverage


In [15]:
"""Check the groups/datasets availability"""
# See all datasets
datasets = get_all_datasets(ckan)
for dataset in datasets:
    print("Dataset name:", dataset["name"], "Title: ", dataset["title"])
# Get all organization groups
org_groups = get_organization_groups(ckan)
for group in org_groups:
    print("ownership name: ", group["name"], "owner_org: ", group["id"])


Dataset name: global-vegetation-carbon-stock-2001-2020 Title:  Vegetation Carbon Stock 2001-2020
ownership name:  integrated-modelling-parnership owner_org:  e7a09a40-27a3-42b0-b265-8e034fe1454a


In [ ]:
"""Get all licenses"""
licenses = get_all_licenses(ckan)
for license in licenses:
    print("License name:", license["title"], " - ID:", license["id"])

In [16]:
"""Prepare the INPUTS"""

"""Local files"""
resources = []

"""Get CSV resources"""
# files_path = r"C:\Users\admin\Downloads\ES"
# file_list = get_csv_file_list(files_path)
# resources = resources + file_list

"""Get OWS resources"""
ows_url = r'https://integratedmodelling.org/aux-geoserver/ows' # https://integratedmodelling.org/dev-geoserver/ows

# Get WCS resources
getcapabilities = r"?service=WCS&request=GetCapabilities"
capabilities_xml = get_url_xml(ows_url + getcapabilities)

regex_pattern = r'wb-nca-kaduna-nasarwa-2015-2020.*' # 'im-nca-data__iucn_.*'

ows_layers = extract_ows_layers(capabilities_xml, regex_pattern)

wcs_layers = describecoverage_or_feature_request(ows_url, ows_layers)
resources = resources + wcs_layers

# WFS resources
# getcapabilities = r"?service=WFS&request=GetCapabilities"
# capabilities_xml = get_url_xml(ows_url + getcapabilities)

# regex_pattern = r'im-data-global-agriculture:crop_polli.*'

# ows_layers = extract_ows_layers(capabilities_xml, regex_pattern)
# wfs_layers = describecoverage_or_feature_request(ows_url, ows_layers)
# resources = resources + wfs_layers
print(resources)

['https://integratedmodelling.org/aux-geoserver/ows?service=WCS&version=2.0.1&request=DescribeCoverage&CoverageId=wb-nca-kaduna-nasarwa-2015-2020__kaduna_corganic_mass_4326_300m_2015', 'https://integratedmodelling.org/aux-geoserver/ows?service=WCS&version=2.0.1&request=DescribeCoverage&CoverageId=wb-nca-kaduna-nasarwa-2015-2020__kaduna_corganic_mass_4326_300m_2020', 'https://integratedmodelling.org/aux-geoserver/ows?service=WCS&version=2.0.1&request=DescribeCoverage&CoverageId=wb-nca-kaduna-nasarwa-2015-2020__kaduna_corganic_soil_mass_4326_300m_2015', 'https://integratedmodelling.org/aux-geoserver/ows?service=WCS&version=2.0.1&request=DescribeCoverage&CoverageId=wb-nca-kaduna-nasarwa-2015-2020__kaduna_corganic_soil_mass_4326_300m_2020', 'https://integratedmodelling.org/aux-geoserver/ows?service=WCS&version=2.0.1&request=DescribeCoverage&CoverageId=wb-nca-kaduna-nasarwa-2015-2020__kaduna_cvegetation_mass_4326_300m_2015', 'https://integratedmodelling.org/aux-geoserver/ows?service=WCS&ver

In [17]:
# If the dataset does not exist. The next areas have to be filled
# El concepto de grupo prefiero que se genere manualmente. Luego asocialo al que quieras.
dataset_name = "ecosystem-accounts-nigeria-2015-2020" #No spaces. This is the name on the URL PATH.
title = "SEEA Ecosystem Accounts for Nasarawa and Kaduna States for 2015 and 2020"
author = "Caterina Gilli, Alessio Bulckaen"
author_email = "caterina.gilli@bc3research.org" # Parece que no se pueden meter mas de un email
maintainer = "Rubén Crespo"
maintainer_email = "ruben.crespo@bc3research.org"
license_id = "cc-by"
description = "this is the description of the dataset"
group = [] # It is a list.
owner_org = "e7a09a40-27a3-42b0-b265-8e034fe1454a" # This is the organization integrated modelling and WILL ALWAYS BE.

In [18]:
dataset_metadata = {
    "name": dataset_name,               # the name of the new dataset
    "title": title,              # the title of the dataset (optional, default: same as name)
    "private": False,         # If True creates a private dataset
    "author": author,             # the name of the dataset’s author (optional)
    "author_email": author_email,       # the email address of the dataset’s author (optional)
    "maintainer": maintainer,         # the name of the dataset’s maintainer (optional)
    "maintainer_email": maintainer_email,   # the email address of the dataset’s maintainer (optional)
    "license_id": license_id,         # the id of the dataset’s license (optional)
    "notes": description,              # a description of the dataset (optional)
    "url": "",                # a URL for the dataset’s source (optional)
    "version": "",            # (optional)
    "state": "active",        # the current state of the dataset (optional, default: 'active')
    "type": "",               # the type of the dataset (optional)
    "resources": [],          # the dataset’s resources, list of resource dictionaries (optional)
    "tags": [],               # the dataset’s tags, list of tag dictionaries (optional)
    "extras": [],             # the dataset’s extras, list of dataset extra dictionaries (optional)
    "relationships_as_object": [],  # list of relationship dictionaries (optional)
    "relationships_as_subject": [], # list of relationship dictionaries (optional)
    "groups": group,             # the groups to which the dataset belongs (optional)
    "owner_org": owner_org           # the id of the dataset’s owning organization (optional)
}

In [19]:
# dataset = None
dataset = get_dataset_by_name(ckan, dataset_name)
if dataset is None:
    # Create the dataset
    new_dataset = create_dataset(ckan, dataset_metadata)
    # Assign the variable
    dataset = new_dataset


Dataset with name 'ecosystem-accounts-nigeria-2015-2020' does not exist.
Create dataset:  ecosystem-accounts-nigeria-2015-2020


In [20]:
for resource in resources[:]:
    if resource.endswith("csv"):
        resource_format = "CSV"
        csv_file = open(resource, mode='rb')

        resource_metadata = {
        'name': os.path.splitext(os.path.basename(resource))[0],
        'description': '',
        'url': '',
        'format': resource_format,
        'package_id': dataset["id"],
        'state': 'active',
        }
        new_resource = create_resource(ckan, resource_metadata, file_obj=csv_file)
        # No es necesario crear un View por ahora, se genera uno automaticamente.
        # view_params = {
        # 'resource_id': new_resource["id"],
        # 'view_type': "grid_view",
        # 'title': "Grid type"
        # }
        # create_view(ckan, view_params)
        csv_file.close() # Hay que cerrarlo siempre
        
    elif "WCS" in resource:    
        """We are going to transform the DescribeFeature to a GetCoverage request"""
        # Get the CoverageID from the sentence
        pattern = r'CoverageId=.*?__(.*?)(&|$)'
        match = re.search(pattern, resource)
        coverage_id = match.group(1)

        """Punto de inflexión: o getcoverage o el describecoverage que ya viene"""
        getcoverage = getcoverage_request(ows_url, coverage_id)

        resource_metadata = {
        'name': coverage_id,
        'description': '',
        'url': getcoverage,
        'datastore_active': False,
        'format': 'WCS',
        'package_id': dataset["id"],
        'state': 'active',
        }
        
        new_resource = create_resource(ckan, resource_metadata, file_obj=None)

        describe_coverage = get_url_xml(resource)
        # Empieza con el View
        getmap_url = getmap_from_describecoverage(ows_url, describe_coverage)

        view_metadata = {
        'description': '',
        'title': coverage_id,
        'resource_id': new_resource["id"],
        'view_type': 'image_view',
        'image_url': getmap_url  # Replace with the URL of your image
        }
        create_view(ckan, view_metadata)


    elif "WFS" in resource:
        print("I did not program this yet")
    else:
        print("wtf")
    

Create resource:  eco_acc_kaduna_carbon_biophysical_supply_by_et_2015-2020
Create resource:  eco_acc_kaduna_sediment_biophysical_supply_by_et_2015-2020
Create resource:  eco_acc_kaduna_soil_organic_carbon_biophysical_supply_by_et_2015-2020
Create resource:  eco_acc_kaduna_vegetation_carbon_biophysical_supply_by_et_2015-2020
Create resource:  eco_acc_nasarawa_carbon_biophysical_supply_by_et_2015-2020
Create resource:  eco_acc_nasarawa__sediment_biophysical_supply_sby_et_2015-2020
Create resource:  eco_acc_nasarawa__soil_organic_carbon_biophysical_supply_by_et_2015-2020
Create resource:  eco_acc_nasarawa__vegetation_carbon_biophysical_supply_by_et_2015-2020
Create resource:  eco_acc_kaduna_corganic_mass_4326_300m_2015
Create resource:  eco_acc_kaduna_corganic_mass_4326_300m_2020
Create resource:  eco_acc_kaduna_corganic_soil_mass_4326_300m_2015
Create resource:  eco_acc_kaduna_corganic_soil_mass_4326_300m_2020
Create resource:  eco_acc_kaduna_cvegetation_mass_4326_300m_2015
Create resour

In [116]:
resource

'https://integratedmodelling.org/aux-geoserver/ows?service=WCS&version=2.0.1&request=DescribeCoverage&CoverageId=im-nca-nigeria__kaduna_corganic_mass_2020'

In [114]:
getmap_url

'https://integratedmodelling.org/aux-geoserver/ows?service=WMS&version=1.1.0&request=GetMap&layers=im-nca-nigeria%3Akaduna_corganic_mass_2020&bbox=6.0934%2C9.0007%2C8.785%2C11.4955&width=998&height=920&srs=EPSG%3A4326&format=image%2Fjpeg'

In [ ]:
"""
'description': '',
'title': 'vcs_2003_global_300m',
'resource_id': '6df149a2-ba88-4a4a-9749-1dd60bb2bc4c',
'view_type': 'image_view',
'id': '41969700-992a-4d69-ac3e-395bd924a5e0',
'package_id': 'b6900b3f-c345-4cae-90a2-809104b0ed76',
'image_url': 'https://data.integratedmodelling.org/geoserver/world_bank/wms?service=WMS&version=1.1.0&request=GetMap&layers=world_bank%3Avcs_2003_global_300m&bbox=-180.0%2C-57.0%2C180.0%2C84.0&width=1152&height=495&srs=EPSG%3A4326&format=image%2Fjpeg'}]
"""

In [73]:
new_resource["id"]

'b848b602-89c0-46e3-8079-b5b38abb1d57'

In [ ]:
delete_dataset(ckan, "9c5227f6-9e20-48ad-8fb4-978c04e98569")

# View types

In CKAN, `view_type` refers to the type of view that you want to create for a resource. These view types determine how the data in the resource will be presented or visualized to users. CKAN provides various view types to accommodate different types of data and visualization needs. Here are some common `view_type` options:

1. **grid_view**: This view type displays tabular data in a grid format, similar to a spreadsheet. It's suitable for displaying datasets with rows and columns of data like CSV.

2. **graph**: This view type allows you to create different types of graphs and charts to visualize data. Graphs could include bar charts, line charts, pie charts, scatter plots, etc.

3. **text_view**: This view type simply displays the raw text content of the resource. It's useful for displaying textual data or files like READMEs, documentation, or plain text files, or CSV.

4. **image_view**: This view type displays images that are embedded within the resource. It's suitable for viewing image files or thumbnails.

5. **map_view**: This view type displays geographical data on an interactive map. It's commonly used for datasets containing geographic coordinates or spatial data.

6. **recline_view**: This view type provides an interactive data exploration interface with features like filtering, sorting, and faceted search. It's useful for exploring and analyzing tabular datasets.

8. **markdown**: This view type allows you to render Markdown content within CKAN. It's useful for displaying formatted text, documentation, or other Markdown-based content.

9. **recline_view**: Provides an interactive grid view with filtering, sorting, and visualization capabilities.

10. **geojson_view**: Displays GeoJSON data on a map.
11. **pdf_view**: Renders a PDF document.

12. **markdown**: Renders Markdown content.

13. **textile**: Renders Textile content.
14. **dcat_rdf_view**: Displays metadata in DCAT RDF format.
15. **json_table_view**: Displays JSON data in a tabular format.

16. **html_view**: Renders HTML content.

17. **javascript_view**: Executes JavaScript code embedded within a view.

18. **pdf_preview**: Provides a preview of a PDF document.

These are just a few examples of `view_type` options available in CKAN. Depending on your CKAN setup and any additional plugins you may have installed, there may be additional view types available. When creating a resource view using `resource_view_create`, you'll need to specify the appropriate `view_type` based on how you want the data to be presented or visualized.
